In [ ]:
import demics as dmc
import cv2
import numpy as np
import matplotlib.pyplot as plt
% matplotlib notebook
import logging
import sys
logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format='%(asctime)s %(name)s [%(levelname)s]:%(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')

In [ ]:
s_train = 1
s_test = 2
train_file = "../data/section_%s.tif"%(str(s_train).zfill(4))
test_file = "../data/section_%s.tif"%(str(s_test).zfill(4)) 
coords_file = "../data/section_%s.{}.txt"%(str(s_train).zfill(4))
patchsize = 30

In [ ]:
# prepare training data for L=2 different classes
I = cv2.imread( train_file, 0 )
labels = ['bg','vessels']
training_patches = []
training_labels = []
for l in labels:
    coords = np.loadtxt(coords_file.format(l))
    patches_l = dmc.extract_patches( I, coords, patchsize )
    training_patches.extend( patches_l )
    training_labels.extend( [labels.index(l)]*len(patches_l) )

In [ ]:
# see the vessel patches
print labels.index("vessels")
vesselpatches = [p for i,p 
                 in enumerate(training_patches) 
                 if training_labels[i]==labels.index("vessels")]
numpatches = len(vesselpatches)
f,axs = plt.subplots( 5, numpatches/5+1, sharex=True,sharey=True)
for i,p in enumerate(vesselpatches):
    axs[i%5,i/5].imshow(p,cmap='gray')
    axs[i%5,i/5].axis('off')
plt.show()

In [ ]:
# Training: train the classifier
classifier = dmc.TextureClassifier()
classifier.train( training_patches, training_labels )

In [ ]:
# collect patches for prediction
I = cv2.imread( test_file, 0 )
gridcoords = dmc.grid_coordinates(I.shape, 10, patchsize/2)
patches = dmc.extract_patches( I, gridcoords, patchsize )

In [ ]:
# prediction: predict labels for N image patches
labels, scores = classifier.predict( patches )
# labels is a Nx1 vector, with elements 0 <= e <=L-1; scores is an NxL array

In [ ]:
# Plot detected vessels
plt.figure()
plt.imshow(I, 'gray')
plt.plot(np.array(gridcoords)[np.where(labels==1)[0]][:,0], 
         np.array(gridcoords)[np.where(labels==1)[0]][:,1], "r.")
plt.show()

In [ ]:
classifier.save(filename="trained_classifier.p")

In [ ]:
classifier2 = dmc.TextureClassifier.load("trained_classifier.p")

In [ ]:
# collect patches for prediction
I = cv2.imread( test_file, 0 )
gridcoords = dmc.grid_coordinates(I.shape, 10, patchsize/2)
patches = dmc.extract_patches( I, gridcoords, patchsize )

In [ ]:
# prediction: predict labels for N image patches
labels, scores = classifier.predict( patches )
# labels is a Nx1 vector, with elements 0 <= e <=L-1; scores is an NxL array

In [ ]:
# Plot detected vessels
plt.figure()
plt.imshow(I, 'gray')
plt.plot(np.array(gridcoords)[np.where(labels==1)[0]][:,0], 
         np.array(gridcoords)[np.where(labels==1)[0]][:,1], "r.")
plt.show()